![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_6 Buttons and LEDs

This short notebook introduces how we access a Phidgets device from a Jupyter notebook.

We will monitor the two pushbuttons on the Phidget starter kit, and use them to control the LEDson that box. 

It is important to run this notebook in the Google Chrome web browser, on a desktop or laptop computer. Opera and MicroSoft Edge should work as well. Unfortunately, other browsers (FireFox, Safari) do not work currently, as they cannot access the USB port that connects to the Phidget device.

Reference code: https://www.phidgets.com/education/learn/getting-started-kit-tutorial/blink-led/
Be sure to look at the JavaScript code.

## Setting up the Phidget device

We will be using the Phidget Starter Kit, as shown in this figure:
<img src="images/KIT4003.jpg" alt="Image of the starter kit"  width = 500 />

Verify these points:
- Get a hold of a Phidgets Starter Kit. It should already be assembled. If not, assemble it now.
    - You will have received instructions with the kit on how to assemble it.
- The Starter Kit has a two buttons and two switches connect. If it does not, find them and attach.
    - We assume the red button is attached to port 0, and the red LED attached to port 1.
    - We assume the green button is attached to port 5, and the green LED attached to port 4.
    - if note, either change yoor wires or chage the "setPortHub" values in the code belwo. 
- Connect the Starter Kit to your computer with a USB cable. 
    - The Kit come with a USB cable, it is attached to the Hub device inside the Starter Kit.
- As mentioned above, make sure you are running this notebook in the Google Chrome browser. 

## Setup the Phidget device

As in the earlier notebooks, the main steps are thus:
1. Set up default values for variables that cross js cells
2. Define and open the USB connection
3. Open the digital inputs (buttons) and outputs (LEDS). There are four in total.
4. Request a connection to the Phidgets device
5. Read values from the buttons, push values to the LEDs 
6. When we are done, close the device

## Step 0
Let's import a few Python libraries that we need. 

In [ ]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1

Set up some variales.


In [ ]:
%%js
window.usbconn = null;
window.inA = null;
window.inB = null;
window.outA = null;
window.outB = null;

## Step 2
Open the USB connection.

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});

require(['phidget22'], function(phidget22) {
    if (window.usbconn == null){
        window.usbconn = new phidget22.USBConnection();
        element.text("Creating a new USB Connection.");
    }
    console.log(window.usbconn)
    
    window.usbconn.connect().catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
    element.append(" Connected.");
});

In [ ]:
## We rest for a bit while the devices connect
sleep(2)

## Check
We check here that the connection is open (should read "true")

In [ ]:
%%js
element.text(window.usbconn.connected);

## Step 3

Open the two input ports and the two output ports. 
- We assume the red button (inA) is attached to port 0, and the red LED (outA) attached to port 1.
- We assume the green button (inB) is attached to port 5, and the green LED (outB) attached to port 4.

If these are different on your device, you will need to change these numbers in the code. 


In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});

require(['phidget22'], function(phidget22) {
        window.inA = new phidget22.DigitalInput();
        window.inA.setHubPort(0);        
        window.inA.setIsHubPortDevice(true); // required for a switch
        window.inA.open();

        window.inB = new phidget22.DigitalInput();
        window.inB.setHubPort(5);        
        window.inB.setIsHubPortDevice(true); // required for a switch
        window.inB.open();

        window.outA = new phidget22.DigitalOutput();
        window.outA.setHubPort(1);        
        window.outA.setIsHubPortDevice(true); // required for an LED
        window.outA.open();

        window.outB = new phidget22.DigitalOutput();
        window.outB.setHubPort(4);        
        window.outB.setIsHubPortDevice(true); // required for an LED
        window.outB.open();

});

In [ ]:
## We rest for a bit while the devices connect.
sleep(2)

## Step 4

We request a connection to the Phidgets devices. 

In [ ]:
%%js
window.usbconn.requestWebUSBDeviceAccess();

## Check

Are the four devices all connected? Expecting "true" for the answers in the following. 

In [ ]:
%%js
element.text("Is Input A connected? " + window.inA.isOpen);
element.append("<p>Is Input B connected? " + window.inB.isOpen);
element.append("<p>Is Output A connected? " + window.outA.isOpen);
element.append("<p>Is Output B connected? " + window.outB.isOpen);

## Step 5

Let's play by turing on and off the LEDs, and reading the buttons.

## Turn LED A on
- run the next cell

In [ ]:
%%js
window.outA.setState(true);

In [ ]:
## Wait a bit so we can see the light
sleep(2)

## Turn LED A off
- run the next cell

In [ ]:
%%js
window.outA.setState(false);

## Turn LED B on
- run the next cell

In [ ]:
%%js
window.outB.setState(true);

In [ ]:
## Wait a bit so we can see the light
sleep(2)

## Turn LED B off 
- run the next cell

In [ ]:
%%js
window.outB.setState(false);

## Is button A pushed on or off?
- run the next cell. Try it both with the button pushed on, or off. 

In [ ]:
%%js
element.text(window.inA.state)

## Is button B pushed on or off?
- run the next cell. Try it both with the button pushed on, or off. 

In [ ]:
%%js
element.text(window.inB.state)

## Connect the button to the LED
- run the next cell. 
- then try pushing the A button. It should turn on the A LED> 

In [ ]:
%%js
window.inA.onStateChange = function (state) {
    window.outA.setState(state);
};


## Connect the B button to a screen button
- run the next to cells
- when you push the B button, the button on the screen should flash on and off

In [ ]:
from ipywidgets import  ToggleButton

buttonB  = ToggleButton(description='Button B')

buttonB

In [ ]:
%%js
window.inB.onStateChange = function (state) {
    window.outB.setState(state);
    let ss = "False";
    if (state) {ss = "True";};
    IPython.notebook.kernel.execute("buttonB.value = " + ss );  
};


## Closing up

When you are all done, you should close up all four devices. Click the button to disconnect.

In [ ]:
def doDisconnect(b):
    display(Javascript("""
    window.inA.close();
    window.inB.close();
    window.outA.close();
    window.outB.close();
    setTimeout(() => { window.usbconn.close(); },1000);
    setTimeout(() => { window.usbconn.delete(); },2000);
    """))
    print("You have disconnected the Phidgets.")

run_button = widgets.Button(
      description = 'Click to disconnect'
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

### Confirm

You can confirm the phidget has been closed by running the following cell. 

If any of them are open (true), try clicking the button above, again.

In [ ]:
%%js
element.text("Is Input A connected? " + window.inA.isOpen);
element.append("<p>Is Input B connected? " + window.inB.isOpen);
element.append("<p>Is Output A connected? " + window.outA.isOpen);
element.append("<p>Is Output B connected? " + window.outB.isOpen);

## And that's it. 

The Phidget device will continue to monitor the two push buttons, display the result on the screen, and light up the appropriate LEDs. .


Good job!

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)